In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = 'https://gomatadors.com/sports/mens-basketball/stats/2024-25'

page = requests.get(url) #requests object

soup = BeautifulSoup(page.text, 'html')

In [3]:
table = soup.find_all('table')[1]
print(table)

<table accordion-table="" class="sidearm-table collapse-on-medium accordion" sortable-table="">
<caption class="hide">Overall Individual Statistics</caption>
<thead class="sidearm-primary">
<tr>
<th class="text-left" rowspan="2" scope="col">#</th>
<th class="text-hide" rowspan="2" scope="col">Player</th>
<th rowspan="2" scope="col">GP</th>
<th rowspan="2" scope="col">GS</th>
<th colspan="2" scope="colgroup">Minutes</th>
<th colspan="3" scope="colgroup">FG</th>
<th colspan="3" scope="colgroup">3PT</th>
<th colspan="3" scope="colgroup">FT</th>
<th colspan="2" scope="colgroup">Scoring</th>
<th colspan="4" scope="colgroup">Rebounds</th>
<th rowspan="2" scope="col">PF</th>
<th rowspan="2" scope="col">AST</th>
<th rowspan="2" scope="col">TO</th>
<th rowspan="2" scope="col">STL</th>
<th rowspan="2" scope="col">BLK</th>
<th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>
</tr>
<tr>
<th scope="col">TOT</th>
<th scope="col">AVG</th>
<th scope="col">FGM</th>
<th sco

In [4]:
#ISSUE: COLUMNS ARE OUT OF ORDER
#After #, GP, GS it goes out of order
    #It jumps to 

#extract the table headers
headers = table.find_all('th', attrs={"scope": "col"})
print(headers)
num_headers = len(headers)

#loop through the headers
table_headers = [title.text.strip() for title in headers]

#Move these headers to the back of the list to fix the order
headers_to_move = ["PF", "AST", "AST/G", "TO", "STL", "BLK", "Bio Link"]

fixed_headers = [h for h in table_headers if h not in headers_to_move]  # Keep other headers in original order
moved_headers = [h for h in table_headers if h in headers_to_move]  # Move these to the end

# Combine lists to get the final order
final_headers = fixed_headers + moved_headers
print(final_headers)

[<th class="text-left" rowspan="2" scope="col">#</th>, <th class="text-hide" rowspan="2" scope="col">Player</th>, <th rowspan="2" scope="col">GP</th>, <th rowspan="2" scope="col">GS</th>, <th rowspan="2" scope="col">PF</th>, <th rowspan="2" scope="col">AST</th>, <th rowspan="2" scope="col">TO</th>, <th rowspan="2" scope="col">STL</th>, <th rowspan="2" scope="col">BLK</th>, <th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>, <th scope="col">FGM</th>, <th scope="col">FGA</th>, <th scope="col">FG%</th>, <th scope="col">3PT</th>, <th scope="col">3PTA</th>, <th scope="col">3PT%</th>, <th scope="col">FTM</th>, <th scope="col">FTA</th>, <th scope="col">FT%</th>, <th scope="col">PTS</th>, <th scope="col">AVG</th>, <th scope="col">OFF</th>, <th scope="col">DEF</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>]
['#', 'Player', 'GP', 'GS', 'TOT', 'AVG', 'FGM', 'FGA', 'FG%', '3PT', '3PTA', '3PT%', 'FTM', 'FT

In [5]:
column_data = table.find_all('tr')
df = pd.DataFrame(columns = final_headers)

for row in column_data[2:]: 
    row_data = row.find_all('td')
    individual_row_data = [data.get_text(separator = " ").strip() for data in row_data]
    individual_row_data[1] = " ".join(individual_row_data[1].split()).split(" ", 1)[-1]
    if len(individual_row_data) > 1:
        words = individual_row_data[1].split()  # Split into words
    if len(words) > 2:  
        individual_row_data[1] = " ".join(words[2:])  # Keep everything after the first two words
        
    print(individual_row_data)
    
    length = len(df)
    df.loc[length] = individual_row_data




['08', '08 Adams Jr., Marcus', '25', '13', '700', '28.0', '159', '299', '.532', '47', '118', '.398', '32', '45', '.711', '397', '15.9', '43', '78', '121', '4.8', '57', '35', '42', '25', '10', 'View Bio']
['12', 'Washington, Scotty', '25', '25', '697', '27.9', '123', '282', '.436', '46', '124', '.371', '79', '91', '.868', '371', '14.8', '13', '62', '75', '3.0', '54', '36', '41', '25', '7', 'View Bio']
['07', 'Jones, Keonte', '25', '25', '772', '30.9', '117', '238', '.492', '12', '30', '.400', '84', '128', '.656', '330', '13.2', '71', '169', '240', '9.6', '73', '107', '66', '50', '37', 'View Bio']
['04', '04 Fuller II, PJ', '25', '25', '617', '24.7', '77', '193', '.399', '26', '97', '.268', '40', '53', '.755', '220', '8.8', '10', '54', '64', '2.6', '63', '77', '58', '25', '6', 'View Bio']
['11', 'Fofana, Mahmoud', '25', '24', '642', '25.7', '71', '117', '.607', '0', '4', '.000', '45', '72', '.625', '187', '7.5', '48', '67', '115', '4.6', '65', '27', '29', '31', '5', 'View Bio']
['21', 'L

In [7]:
df


,#,Player,GP,GS,TOT,AVG,FGM,FGA,FG%,3PT,...,OFF,DEF,TOT,AVG,PF,AST,TO,STL,BLK,Bio Link
0,08,"08 Adams Jr., Marcus",25,13,700,28.0,159,299,.532,47,...,43,78,121,4.8,57,35,42,25,10,View Bio
1,12,"Washington, Scotty",25,25,697,27.9,123,282,.436,46,...,13,62,75,3.0,54,36,41,25,7,View Bio
2,07,"Jones, Keonte",25,25,772,30.9,117,238,.492,12,...,71,169,240,9.6,73,107,66,50,37,View Bio
3,04,"04 Fuller II, PJ",25,25,617,24.7,77,193,.399,26,...,10,54,64,2.6,63,77,58,25,6,View Bio
4,11,"Fofana, Mahmoud",25,24,642,25.7,71,117,.607,0,...,48,67,115,4.6,65,27,29,31,5,View Bio
5,21,"Lewis, Grady",25,13,338,13.5,67,118,.568,0,...,46,54,100,4.0,61,14,18,6,5,View Bio
6,00,"Beard, Tyler",25,0,477,19.1,45,117,.385,18,...,9,61,70,2.8,34,76,42,19,3,View Bio
7,23,"Brinson, Jordan",21,0,312,14.9,18,64,.281,7,...,15,39,54,2.6,30,36,16,14,1,View Bio
8,33,"Martindale, Judson",18,0,162,9.0,16,37,.432,6,...,6,14,20,1.1,21,5,7,3,3,View Bio
9,06,"Ndumanya, Festus",22,0,133,6.0,23,51,.451,0,...,12,14,26,1.2,32,4,11,4,2,View Bio
